In [ ]:
import geopyspark as gps
from pyspark import SparkContext

In [ ]:
conf = gps.geopyspark_conf(appName="gps-osm-ingest", master='yarn')
conf.set("spark.hadoop.yarn.timeline-service.enabled", False)
conf.set('spark.ui.enabled', True)
conf.set('spark.default.parallelism', 128)
conf.set('spark.executor.memory', '9500M')
conf.set('spark.executor.cores', 8)
conf.set('spark.yarn.executor.memoryOverhead', '1500M')
conf.set('spark.master.memory', '9500M')
conf.set('spark.yarn.driver.memoryOverhead', '1500M')
conf.set('spark.dynamicAllocation.enabled', True)
conf.set('spark.shuffle.service.enabled', True)
sc = SparkContext(conf=conf)

In [ ]:
uri = "s3://geotrellis-test/dg-srtm/california.orc"

In [ ]:
features = gps.osm_reader.read(uri)
lines = features.get_line_features_rdd()
polygons = featuers.get_polygon_features_rdd()

In [ ]:
filtered_lines = lines.filter(lambda feature: 'highway' in feature.properties.tags or 'waterway' in feature.properties.tags)

In [ ]:
mapped_lines = filtered_lines.map(lambda feature: gps.Feature(feature.geometry, gps.CellValue(2, 2)))
mapped_polygons = polygons.map(lambda feature: gps.Feature(feature.geometry, gps.CellValue(1, 1)))

In [ ]:
unioned_features = sc.union((mapped_lines, mapped_polygons))

In [ ]:
rasterized = gps.rasterize_features(unioned_features,
                                    4326,
                                    13,
                                    cell_type=gps.CellType.INT8,
                                    num_partitions=512)

In [ ]:
tiled_layer = rasterized.tile_to_layout(gps.GlobalLayout(zoom=13, tile_size=256), target_crs=4326)
pyramid = tiled_layer.pyramid()

In [ ]:
for layer in pyrmaid.levels.values():
    gps.write("s3://geotrellis-test/dg-osm-test", "gps-osm-ingest", layer)